Note : in the imports.core.py file, unhighlight # 29-31, 24 and download fastprogress

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [12]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

sns.set_style('darkgrid')

In [3]:
file = '../data/bluebook-for-bulldozers/Train.csv'

In [68]:
df_raw = pd.read_csv(file, parse_dates=['saledate'], low_memory=False)

In [69]:
df_raw.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   401125 non-null  int64         
 1   SalePrice                 401125 non-null  int64         
 2   MachineID                 401125 non-null  int64         
 3   ModelID                   401125 non-null  int64         
 4   datasource                401125 non-null  int64         
 5   auctioneerID              380989 non-null  float64       
 6   YearMade                  401125 non-null  int64         
 7   MachineHoursCurrentMeter  142765 non-null  float64       
 8   UsageBand                 69639 non-null   object        
 9   saledate                  401125 non-null  datetime64[ns]
 10  fiModelDesc               401125 non-null  object        
 11  fiBaseModel               401125 non-null  object        
 12  fi

In [7]:
df_raw.describe()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter
count,4.011250e+05,401125.000000,4.011250e+05,401125.000000,401125.000000,380989.000000,401125.000000,1.427650e+05
mean,1.919713e+06,31099.712848,1.217903e+06,6889.702980,134.665810,6.556040,1899.156901,3.457955e+03
std,9.090215e+05,23036.898502,4.409920e+05,6221.777842,8.962237,16.976779,291.797469,2.759026e+04
min,1.139246e+06,4750.000000,0.000000e+00,28.000000,121.000000,0.000000,1000.000000,0.000000e+00
25%,1.418371e+06,14500.000000,1.088697e+06,3259.000000,132.000000,1.000000,1985.000000,0.000000e+00
50%,1.639422e+06,24000.000000,1.279490e+06,4604.000000,132.000000,2.000000,1995.000000,0.000000e+00
75%,2.242707e+06,40000.000000,1.468067e+06,8724.000000,136.000000,4.000000,2000.000000,3.025000e+03
max,6.333342e+06,142000.000000,2.486330e+06,37198.000000,172.000000,99.000000,2013.000000,2.483300e+06


In [70]:
df_raw['SalePrice'] = np.log(df_raw['SalePrice'])

In [84]:
X, y = df_raw.drop('SalePrice', axis=1), df_raw['SalePrice'].copy()

In [10]:
rf = RandomForestRegressor(max_samples=10000).fit(X, y)

ValueError: could not convert string to float: 'Low'

In [72]:
# extracting features fromthe date attribute
def add_date(df, name):
    fld = df[name]
    target = re.sub('[Dd]ate$', '', name)
    for n in ['Year', 'Month', 'Day']:
        df[target+n] = getattr(fld.dt, n.lower())
    df.drop(name, axis=1, inplace=True)
add_date(df_raw, 'saledate')

In [73]:
df_raw

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleDay
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,Low,521D,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,2006,11,16
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,Low,950FII,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,2004,3,26
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,High,226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004,2,26
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,5,19
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009,7,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,9.259131,1840702,21439,149,1.0,2005,NaN,NaN,35NX2,...,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2011,11,2
401121,6333337,9.305651,1830472,21439,149,1.0,2005,NaN,NaN,35NX2,...,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2011,11,2
401122,6333338,9.350102,1887659,21439,149,1.0,2005,NaN,NaN,35NX2,...,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2011,11,2
401123,6333341,9.104980,1903570,21435,149,2.0,2005,NaN,NaN,30NX,...,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2011,10,25


In [18]:
df_raw['UsageBand'] = pd.Categorical(df_raw['UsageBand']).codes+1
df_raw['UsageBand']

0         2
1         2
2         4
3         4
4         3
         ..
401120    1
401121    1
401122    1
401123    1
401124    1
Name: UsageBand, Length: 401125, dtype: int8

In [74]:
df_raw.isnull().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    258360
UsageBand                   331486
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             137191
fiModelSeries               344217
fiModelDescriptor           329206
ProductSize                 210775
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                296764
Enclosure                      325
Forks                       209048
Pad_Type                    321991
Ride_Control                252519
Stick                       321991
Transmission                217895
Turbocharged                321991
Blade_Extension             375906
Blade_Width         

In [86]:
# Changing object to categorical data types
for column, content in df_raw.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_raw[column] = content.astype('category').cat.as_ordered()

In [87]:
# dealing with null values and filling the the values of missing numeric type with their median value       
for column, content in df_raw.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():                         
            df_raw[column+'_na'] = pd.isnull(content)
            df_raw[column] = content.fillna(content.median())

In [88]:
df_raw

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Undercarriage_Pad_Width_na,Stick_Length_na,Thumb_na,Pattern_Changer_na,Grouser_Type_na,Backhoe_Mounting_na,Blade_Type_na,Travel_Controls_na,Differential_Type_na,Steering_Controls_na
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,2,521D,...,True,True,True,True,True,True,True,True,False,False
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,2,950FII,...,True,True,True,True,True,True,True,True,False,False
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,1,226,...,True,True,True,True,True,True,True,True,True,True
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,1,PC120-6E,...,True,True,True,True,True,True,True,True,True,True
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,3,S175,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,9.259131,1840702,21439,149,1.0,2005,0.0,0,35NX2,...,False,False,False,False,False,True,True,True,True,True
401121,6333337,9.305651,1830472,21439,149,1.0,2005,0.0,0,35NX2,...,False,False,False,False,False,True,True,True,True,True
401122,6333338,9.350102,1887659,21439,149,1.0,2005,0.0,0,35NX2,...,False,False,False,False,False,True,True,True,True,True
401123,6333341,9.104980,1903570,21435,149,2.0,2005,0.0,0,30NX,...,False,False,False,False,False,True,True,True,True,True


In [89]:
# Dealing with categorical values
for column, content in df_raw.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():                                     
            df_raw[column+'_na'] = pd.isnull(content)
            df_raw[column] = pd.Categorical(content).codes+1

In [90]:
df_raw

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Undercarriage_Pad_Width_na,Stick_Length_na,Thumb_na,Pattern_Changer_na,Grouser_Type_na,Backhoe_Mounting_na,Blade_Type_na,Travel_Controls_na,Differential_Type_na,Steering_Controls_na
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,2,521D,...,True,True,True,True,True,True,True,True,False,False
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,2,950FII,...,True,True,True,True,True,True,True,True,False,False
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,1,226,...,True,True,True,True,True,True,True,True,True,True
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,1,PC120-6E,...,True,True,True,True,True,True,True,True,True,True
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,3,S175,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,9.259131,1840702,21439,149,1.0,2005,0.0,0,35NX2,...,False,False,False,False,False,True,True,True,True,True
401121,6333337,9.305651,1830472,21439,149,1.0,2005,0.0,0,35NX2,...,False,False,False,False,False,True,True,True,True,True
401122,6333338,9.350102,1887659,21439,149,1.0,2005,0.0,0,35NX2,...,False,False,False,False,False,True,True,True,True,True
401123,6333341,9.104980,1903570,21435,149,2.0,2005,0.0,0,30NX,...,False,False,False,False,False,True,True,True,True,True


In [121]:
display(df_raw.iloc[4:, 9::2].tail(20).T)

,401105,401106,401107,401108,401109,401110,401111,401112,401113,401114,401115,401116,401117,401118,401119,401120,401121,401122,401123,401124
fiModelDesc,654,654,654,654,654,654,654,654,654,654,654,654,654,654,482,656,656,656,482,482
fiSecondaryDesc,101,101,101,101,101,101,101,101,101,101,101,101,101,101,106,106,106,106,106,106
fiModelDescriptor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
fiProductClassDesc,16,16,16,16,16,16,16,16,16,16,16,16,16,16,12,16,16,16,12,12
ProductGroup,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Drive_System,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Forks,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Ride_Control,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Transmission,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Blade_Extension,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [133]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 95 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SalesID                      401125 non-null  int64  
 1   SalePrice                    401125 non-null  float64
 2   MachineID                    401125 non-null  int64  
 3   ModelID                      401125 non-null  int64  
 4   datasource                   401125 non-null  int64  
 5   auctioneerID                 401125 non-null  float64
 6   YearMade                     401125 non-null  int64  
 7   MachineHoursCurrentMeter     401125 non-null  float64
 8   UsageBand                    401125 non-null  int8   
 9   fiModelDesc                  401125 non-null  int16  
 10  fiBaseModel                  401125 non-null  int16  
 11  fiSecondaryDesc              401125 non-null  int16  
 12  fiModelSeries                401125 non-null  int8   
 13 

In [134]:
X, y = df_raw.drop('SalePrice', axis=1), df_raw['SalePrice'].copy()

In [137]:
rf = RandomForestRegressor(max_samples=10000, n_jobs=-1).fit(X, y)
rf.score(X, y)

0.8752234282942218

##  Splitting the data into train and validation sets

In [165]:
test_size = 12000
train_size = len(X) - test_size

def split_data(df, size):
    #splits data in train and validation sets
    return df[:size].copy(), df[size:].copy()

In [166]:
X_train, X_valid = split_data(X, train_size)
y_train, y_test = split_data(y, train_size)

In [167]:
X_train.shape, X_valid.shape, y_train.shape

((389125, 94), (12000, 94), (389125,))

In [169]:
rf= RandomForestRegressor(max_samples=20000, n_jobs=-1).fit(X_train, y_train)
rf.score(X_train, y_train)

0.893599595710952

In [162]:
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.30)

In [163]:
rf= RandomForestRegressor(max_samples=10000, n_jobs=-1).fit(X_trn, y_trn)
rf.score(X_trn, y_trn)

0.8776068538967361

In [174]:
import os
os.makedirs('tmp', exits_ok=True)
df_raw.to_feather('tmp/bulldozer-raw1')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/eclipse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-174-85505530e0ab>", line 2, in <module>
    os.makedirs('tmp', exits_ok=True)
TypeError: makedirs() got an unexpected keyword argument 'exits_ok'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/eclipse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/eclipse/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1148, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_

TypeError: makedirs() got an unexpected keyword argument 'exits_ok'

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in callback <bound method AutoreloadMagics.post_execute_hook of <autoreload.AutoreloadMagics object at 0x7f50e55739d0>> (for post_execute):
Traceback (most recent call last):
  File "/home/eclipse/anaconda3/lib/python3.7/site-packages/IPython/core/events.py", line 88, in trigger
    func(*args, **kwargs)
  File "/home/eclipse/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 538, in post_execute_hook
    _, pymtime = self._reloader.filename_and_mtime(sys.modules[modname])
  File "/home/eclipse/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 184, in filename_and_mtime
    if not hasattr(module, '__file__') or module.__file__ is None:
  File "/home/eclipse/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/home/eclipse/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  Fi

ImportError: cannot import name 'AutoGraphError' from 'tensorflow.python.autograph.impl.api' (/home/eclipse/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/autograph/impl/api.py)